# 1. Ý tưởng

- Công đoạn đầu tiên của quy trình Phân tích dữ liệu (Data Analysis) là thu thập dữ liệu. Để khởi đầu cho đồ án này, nhóm lựa chọn chủ đề thời tiết, một lĩnh vực gắn liền với đời sống con người và có ảnh hưởng sâu rộng đến nhiều ngành nghề.

- Thời gian lên kế hoạch: Cuối tháng 10 năm 2025 — giai đoạn Việt Nam đang bước vào mùa mưa, với lượng mưa lớn trên khắp cả nước. Đặc biệt, thời điểm này vừa xảy ra trận lũ lịch sử tại Huế, gây thiệt hại nặng nề về người và tài sản.

- Từ sự tò mò khoa học và mong muốn đóng góp vào công tác dự báo, phân tích thời tiết, nhóm quyết định thực hiện đề tài thu thập và phân tích dữ liệu thời tiết, trong đó hiện tượng mưa được chọn làm tâm điểm nghiên cứu.

- Cơ sở nghiên cứu: Sau khi tìm hiểu, nhóm lựa chọn sử dụng API từ Open-Meteo (https://open-meteo.com/
) để thu thập dữ liệu thời tiết tại khu vực Thành phố Hồ Chí Minh trong năm 2024. Dữ liệu bao gồm các đặc trưng như: nhiệt độ, độ ẩm, lượng mưa, tốc độ gió, hướng gió, độ che phủ mây, áp suất khí quyển, v.v.

# 2. Quá trình



### A. Nguồn data:
| `Nguồn: Open-Meteo API`

- Open-Meteo là nền tảng cung cấp dữ liệu thời tiết miễn phí và không yêu cầu xác thực API key. Bên cạnh dữ liệu dự báo, Open-Meteo còn cung cấp dữ liệu lịch sử (historical / reanalysis) thông qua `Archive API`, cho phép truy xuất thông tin thời tiết thực tế đã xảy ra theo giờ hoặc theo ngày.

- Dữ liệu được tái phân tích (reanalysis) từ nhiều nguồn quan trắc và mô hình khí tượng toàn cầu uy tín như ICON, GFS, và ECMWF, đảm bảo độ tin cậy và tính đại diện cao. Đây là nguồn này đặc biệt phù hợp cho mục tiêu nghiên cứu thời tiết thực tế tại Việt Nam, chẳng hạn như phân tích các yếu tố khí hậu (nhiệt độ, lượng mưa, độ ẩm, tốc độ gió, áp suất, độ che phủ mây,...) trong giai đoạn có mưa lớn trên diện rộng.
 
### B. Quá trình thu thập:

- Khảo sát và lựa chọn nguồn dữ liệu:
Tiến hành tìm hiểu các nguồn dữ liệu thời tiết công khai, đánh giá tính sẵn có, độ tin cậy và định dạng dữ liệu trước khi lựa chọn Open-Meteo làm nguồn chính.

- Thu thập dữ liệu qua API:
Sử dụng phương pháp gọi API (API Call) để truy xuất dữ liệu thời tiết theo giờ của khu vực Thành phố Hồ Chí Minh và các vùng lân cận.
Các thông tin được thu thập bao gồm: thời gian, nhiệt độ, độ ẩm, lượng mưa, tốc độ gió, hướng gió, độ che phủ mây, áp suất khí quyển, v.v.

- Lưu trữ và xử lý:
Dữ liệu sau khi thu thập được chuyển đổi sang định dạng CSV để thuận tiện cho việc tiền xử lý, trực quan hóa và phân tích thống kê trong các bước tiếp theo.

In [1]:
import requests
import csv
import time
import random

# Khu vực quanh TP.HCM
center_lat, center_lon = 10.7769, 106.7009  # trung tâm HCM
lat_range = (10.5, 11.0)    # khoảng vĩ độ
lon_range = (106.3, 107.0)  # khoảng kinh độ
# Tạo 3 tọa độ ngẫu nhiên trong khu vực
num_points = 3
coords = []
for _ in range(num_points):
    lat = round(random.uniform(*lat_range), 4)
    lon = round(random.uniform(*lon_range), 4)
    coords.append((lat, lon))

print("Các tọa độ được chọn quanh HCM:")
for i, (lat, lon) in enumerate(coords, 1):
    print(f"{i:2d}: ({lat}, {lon})")

# Thời gian lấy dữ liệu
start_date = "2024-01-01"
end_date   = "2025-10-30"

output_file = "data/hcm_weather_2024.csv"
header = [
    'latitude','longitude','datetime',
    'temperature','humidity','precipitation',
    'windspeed','winddirection','cloudcover','pressure',
    'dewpoint'
]

with open(output_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    
# Gọi API lấy dữ liệu 
for (lat, lon) in coords:
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": "temperature_2m,relativehumidity_2m,precipitation,"
                  "windspeed_10m,winddirection_10m,cloudcover,pressure_msl,"
                  "dewpoint_2m",
        "timezone": "Asia/Ho_Chi_Minh"
    }

    try:
        resp = requests.get(url, params=params, timeout=60)
        resp.raise_for_status()
        data = resp.json()
    except Exception as e:
        print(f"Lỗi tại ({lat},{lon}): {e}")
        continue

    hourly = data.get("hourly", {})
    times = hourly.get("time", [])
    temps = hourly.get("temperature_2m", [])
    hums  = hourly.get("relativehumidity_2m", [])
    precs = hourly.get("precipitation", []) #Lượng mưa nè
    winds = hourly.get("windspeed_10m", [])
    windd = hourly.get("winddirection_10m", [])
    clouds= hourly.get("cloudcover", [])
    press = hourly.get("pressure_msl", [])
    dews = hourly.get("dewpoint_2m", [])

    with open(output_file, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for i in range(len(times)):
            writer.writerow([
                lat, lon, times[i],
                temps[i], hums[i], precs[i],
                winds[i], windd[i], clouds[i], press[i]
            ])
    # Tạm dừng tránh overload server
    time.sleep(1)

print("Đã lưu: data/hcm_weather_2024.csv")


Các tọa độ được chọn quanh HCM:
 1: (10.5359, 106.8825)
 2: (10.9189, 106.4479)
 3: (10.5782, 106.788)
Đã lưu: data/hcm_weather_2024.csv
